In [3]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
import pandas as pd
import numpy as np

import plotly.graph_objects as go
from plotly.subplots import make_subplots

from helper import groupby_consecutive

import sys
sys.path.append('..')
from plotly_resampler import FigureResampler
from plotly_resampler.downsamplers import EveryNthPoint, LTTB

/home/jonas/git/gIDLab/plotly-dynamic-resampling/.venv/lib/python3.9/site-packages/dash_bootstrap_components/_table.py:5: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html


# The exmample from the `.gif`

In [5]:
df_gusb = pd.read_parquet("data/df_gusb.parquet", engine='fastparquet')
df_data_pc = pd.read_parquet("data/df_pc_test.parquet", engine='fastparquet')

n = 110_000_000
np_series = np.array(
    (3 + np.sin(np.arange(n) / 200_000) + np.random.randn(n) / 10) * np.arange(n) / 100_000,
    dtype=np.float32,
)
x = np.arange(len(np_series))

In [6]:
fig = FigureResampler(
    make_subplots(
        rows=2,
        cols=2,
        specs=[[{}, {}], [{"colspan": 2}, None]],
        subplot_titles=("GUSB swimming pool", "Generated sine", "Power consumption"),
        vertical_spacing=0.12,
    ),
    default_n_shown_samples=1_000,
    verbose=False,
)


# ------------ swimming pool data -----------
df_gusb_pool = df_gusb[df_gusb.zone == "zwembad"]
df_gusb_pool = df_gusb_pool[df_gusb_pool["aantal aanwezigen"] < 3_000].last("4D")
fig.add_trace(
    go.Scattergl(
        x=df_gusb_pool.index,
        y=df_gusb_pool["aantal aanwezigen"].astype("uint16"),
        mode="markers",
        marker_size=5,
        name="occupancy",
        showlegend=True,
    ),
    hf_hovertext="mean last hour: "
    + df_gusb_pool["aantal aanwezigen"].rolling("1h").mean().astype(int).astype(str),
    downsampler=EveryNthPoint(interleave_gaps=False),
    row=1,
    col=1,
)
fig.update_yaxes(title_text="Occupancy", row=1, col=1)


# ----------------- generated sine -----------
fig.add_trace(
    go.Scattergl(name="sin", line_color="#26b2e0"),
    hf_x=x,
    hf_y=np_series,
    row=1,
    col=2,
)

# ------------- Power consumption data -------------
df_data_pc = df_data_pc.last("190D")
for i, c in enumerate(df_data_pc.columns):
    fig.add_trace(
        go.Scattergl(
            name=f"room {i+1}",
        ),
        hf_x=df_data_pc.index,
        hf_y=df_data_pc[c],
        row=2,
        col=1,
        downsampler=LTTB(interleave_gaps=True),
    )

fig.update_layout(height=600)
fig.update_yaxes(title_text="Watt/hour", row=2, col=1)
fig.update_layout(
    title="<b>Plotly-Resampler demo</b>",
    title_x=0.5,
    legend_traceorder="normal",
)

fig.show_dash(mode="external", debug=True, port=9033)


Dash app running on http://127.0.0.1:9033/


# Skin conductance example

In [7]:
df_gsr = pd.read_parquet('data/processed_gsr.parquet', engine='fastparquet')

In [7]:
fig = FigureResampler(
    make_subplots(
        rows=2,
        cols=1,
        specs=[[{"secondary_y": True}], [{}]],
        shared_xaxes=True,
    ),
    default_n_shown_samples=1_000,
    verbose=False,
)
fig.update_layout(height=600)


# -------------------------------- ROW 1 --------------------------------
# Add the skin conductance signals
for c in ["EDA", "EDA_lf_cleaned", "EDA_lf_cleaned_tonic"]:
    fig.add_trace(go.Scattergl(name=c), hf_x=df_gsr.index, hf_y=df_gsr[c], row=1, col=1)


# Show the Skin Conductance Response Peaks (SCR)
df_peaks = df_gsr[df_gsr["SCR_Peaks_neurokit_reduced_acc"] == 1]
fig.add_trace(
    trace=go.Scattergl(
        x=df_peaks.index,
        y=df_peaks["EDA_lf_cleaned"],
        visible="legendonly",
        mode="markers",
        marker_symbol="cross",
        marker_size=15,
        marker_color="red",
        name="SCR peaks",
    ),
    # Set limit_to_view to true so that the peaks dissapear when out-of view-range 
    # and thus not disturb the autoscale :)
    limit_to_view=True,
)


# Display the Sking conductance Signal Quality As background 
df_grouped = groupby_consecutive(df_gsr["EDA_SQI"])
df_grouped["EDA_SQI"] = df_grouped["EDA_SQI"].map(bool)
df_grouped["good_sqi"] = df_grouped["EDA_SQI"].map(int)
df_grouped["bad_sqi"] = (~df_grouped["EDA_SQI"]).map(int)
for sqi_col, col_or in [
    ("good_sqi", "#2ca02c"),
    ("bad_sqi", "#d62728"),
]:
    fig.add_trace(
        go.Scattergl(
            x=df_grouped["start"],
            y=df_grouped[sqi_col],
            mode="lines",
            line_width=0,
            fill="tozeroy",
            fillcolor=col_or,
            opacity=0.1 if "good" in sqi_col else 0.2,
            line_shape="hv",
            name=sqi_col,
            showlegend=False,
        ),
        max_n_samples=len(df_grouped) + 1,
        downsampler=EveryNthPoint(interleave_gaps=False),
        limit_to_view=True,
        secondary_y=True,
    )


# -------------------------------- ROW 2 --------------------------------
# show the phasic EDA component
fig.add_trace(
    go.Scattergl(name="EDA_Phasic", visible="legendonly"),
    hf_x=df_gsr.index,
    hf_y=df_gsr["EDA_Phasic"],
    row=2,
    col=1,
)

fig.show_dash(mode="external", port=9022)

Dash app running on http://127.0.0.1:9022/
